# Proyecto Final
## Procesamiento de Datos
##### Gonzalo Fernández Suárez

---

1) Crear un CALLER para descargar noticias de Cryptomonedas (2.0 puntos de 10)

1.1 - Conectarse a la API de twitters para descargar tweets sobre Cryptomonedas (0.5 puntos de 10).

1.2 - Conectarse a la API de Gdelt para descargar noticias sobre Cryptomonedas (0.5 puntos de 10).

1.3 - Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas (1.0 puntos de 10).

2) Crear un SERVER de Spark Streamming (1 puntos de 10)

3) Hacer un analisis de los mensajes en directo en clase, con ventana de tiempo (1 puntos de 10)

4) Aplicar un modelo de analisis de sentimiento (2.5 puntos de 10)

4.1 - Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT (+0.5 puntos de 10).

4.2 - Aplicar todos los modelo: Vader Sentiment Analysis, textblob, flair y FinBERT (+0.5 puntos de 10).

4.3 - Crear un modelo de machine learning propio y aplicar dicho modelo a los mensajes (+1.5 puntos de 10).

Utilizando los siguientes datos etiquetados manualmente:
20220421_FULLDATASET_FOR_UTAD.csv

4.3.1 - Utilizar news como texto y final_manual_labelling como variable target para vuestro modelo!

 

5) Guardar los datos del SERVER en SQL marcados por tiempo (1.0 puntos de 10)

6) Presentar gráficas de evolución de los sentimiento y comparativa (1.5 puntos de 10)

6.1 - Presentar gráficas de evolución de los sentimiento, uno de los modelo como Vader Sentiment Analysis, textblob y flair (+1.0 puntos de 10).

6.2 - Presentar gráficas de evolución de los sentimiento de todos los modelo: Vader Sentiment Analysis, textblob y flair (+0.5 puntos de 10).

7) Incluir en la comparativa tu propio modelo, y ganas nota máxima si tu modelo es mejeor que los anteriores (+1.0 puntos de 10).

[Tutorial instalacion Spark en Windows](https://phoenixnap.com/kb/install-spark-on-windows-10).
El link de Java esta mal, descargar JDK no JRE

[Runear spark](https://aamargajbhiye.medium.com/apache-spark-setup-a-multi-node-standalone-cluster-on-windows-63d413296971)
```
cmd >> cd C:\Users\User\Desktop\U-Tad\Procesamiento de Datos\spark-3.2.1-bin-hadoop3.2

# Levantar Master Node
cmd >> bin\spark-class org.apache.spark.deploy.master.Master --host localhost
```
http://DESKTOP-GFS:8080

```
# Poner a escuchar al Worker Node
cmd >> bin\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost
```

# Aplicacion PYTHON

Nota: Este proyecto está desarrollado enteramente en un sistema **Windows**. Las máquinas virtuales Ubuntu me han dado malas experiencias petandome el ordenador o bien rompiendose dañando toda la informacion que habia en ellas.

## 1.3 Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas

## 2 Crear un SERVER de Spark Streamming

In [5]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [30]:
# Inicializamos el Contexto
sc = SparkContext("local[2]" , "NetworkWordCount" ) # Master y nombre de la app
ssc = StreamingContext(sc, 10) # Contexto de streaming, contexto de spark y cada cuanto se repite el proceso

# Inicializamos el socket => Comunicamos el origen de los datos
lines = ssc.socketTextStream("localhost", 9090) # Host, Puerto

## 3 Hacer un analisis de los mensajes en directo, con ventana de tiempo

Retocar comentarios del siguiente bloque de codigo

In [5]:
# Como vamos a procesar los datos haciendo duplas con las palabras recibidas y el numero de veces que se repiten
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word,1))
wordCounts = pairs.reduceByKey(lambda x,y: x+y)

wordCounts.pprint()

Ahora en la consola de comandos nos conectamos, usando netCat, a "localhost" en el puerto 9090. 

[Instalación netCat en Windows](https://programmerclick.com/article/5414801604/)

Abrimos la consola de comandos y ejecutamos:
```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```
Ahora desde la cmd podemos enviar información en tiempo real a Spark para que la analice.

In [6]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

-------------------------------------------
Time: 2022-05-01 18:23:20
-------------------------------------------
('Hola', 1)

-------------------------------------------
Time: 2022-05-01 18:23:30
-------------------------------------------
('Soy', 1)
('Gonzalo', 1)
('probando', 1)
('Estoy', 1)

-------------------------------------------
Time: 2022-05-01 18:23:40
-------------------------------------------
('como', 1)
('Spark', 1)
('funciona', 1)
('Streaming', 1)

-------------------------------------------
Time: 2022-05-01 18:23:50
-------------------------------------------
('mensajes', 1)
('en', 1)
('Con', 1)
('directo', 1)

-------------------------------------------
Time: 2022-05-01 18:24:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:10
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:20
-------------------------------------------



In [ ]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

## 4 Aplicar un modelo de analisis de sentimiento a los mensajes

### 4.1 Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT

##### Vader Sentiment Analysis

```
>> pip install vaderSentiment
```

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
# Generamos un nuevo Server de Spark Streaming para el Vader Sentiment Analysis
sc = SparkContext("local[2]" , "NetworkVaderAnalysis" ) 
ssc = StreamingContext(sc, 10) 
lines = ssc.socketTextStream("localhost", 9090) 

In [3]:
# Procesamiento de los datos
vader = SentimentIntensityAnalyzer() # Inicializamos el anlazidador de Vader
# Genero Tuplas con la Linea recibida y su valor 'compound' del Vader Sentiment Analysis
linesAnalyzed = lines.map(lambda line: (line,vader.polarity_scores(line)['compound']))
# Pinto el resultado
linesAnalyzed.pprint()

```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```

In [4]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

-------------------------------------------
Time: 2022-05-03 13:50:50
-------------------------------------------
('hello, good morning', 0.4404)

-------------------------------------------
Time: 2022-05-03 13:51:00
-------------------------------------------
('my name is Gonzalo', 0.0)

-------------------------------------------
Time: 2022-05-03 13:51:10
-------------------------------------------
('im a software lover', 0.5859)

-------------------------------------------
Time: 2022-05-03 13:51:20
-------------------------------------------

-------------------------------------------
Time: 2022-05-03 13:51:30
-------------------------------------------
('and rigth now im testing spark streaming doing a sentiment analysis of my text', 0.2263)

-------------------------------------------
Time: 2022-05-03 13:51:40
-------------------------------------------

-------------------------------------------
Time: 2022-05-03 13:51:50
-------------------------------------------
('i hope this

In [ ]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

### 4.2 Aplicar todos los modelo: Vader Sentiment Analysis, textblob, flair y FinBERT

Apliquemos al mismo tiempo todos los modelos de Sentiment Analysis a las lineas de texto.

```
>> pip install vaderSentiment
>> pip install textblob
>> pip install sentencepiece
>> pip install flair   
>> pip install transformers
```

In [1]:
!conda install -c bioconda flair

^C


In [ ]:
!conda install -c bioconda/label/broken flair

In [ ]:
!pip install vaderSentiment
!pip install textblob
!pip install sentencepiece
!pip install flair   
!pip install transformers

In [37]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# TextBlob
from textblob import TextBlob
# Flair
from flair.models import TextClassifier
from flair.data import Sentence
# FinBert
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [69]:
# Generamos un nuevo Server de Spark Streaming para el Vader Sentiment Analysis
# spark = SparkSession.builder.appName('Sentiment Analysis').master('local').getOrCreate()
sc = SparkContext("local[2]" , "NetworkVaderAnalysis" ) 
ssc = StreamingContext(sc, 20) 
lines = ssc.socketTextStream("localhost", 9090) 

In [70]:
# Prerequisitos
# Vader
vader = SentimentIntensityAnalyzer() # Inicializamos el anlazidador de Vader

# Flair
classifier = TextClassifier.load('en-sentiment') # Inicializamos el clasificador de Flair en ingles
def apply_fair(x):  # Definimos la funcion que aplicará el analisis de Flair a cada texto
    sentence = Sentence(x)
    classifier.predict(sentence)
    return sentence.labels[0].to_dict()['confidence']

# FinBert
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
def apply_finbert(x):
#    return (0,1,2)
    inputs = tokenizer([x], padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=1) 
    return predictions[:, 0].tolist()[0], predictions[:, 1].tolist()[0], predictions[:, 2].tolist()[0][0]


2022-05-05 11:48:40,447 loading file C:\Users\gonef\.flair\models\sentiment-en-mix-distillbert_4.pt


In [62]:
print(apply_fair('hello, today is a nice day'))
apply_finbert('hello, today is a nice day')[0]

0.8923711180686951


0.7736659049987793

In [71]:
## Como vamos a procesar los datos
# Generamos tuplas con la linea analizada y el resultado obtenido de cada tecnica de Sentiment Analysis aplicada
vaderPair = lines.map(lambda line: (line,vader.polarity_scores(line)['compound']) ) # Vader
textBlobPair = lines.map(lambda line : (line,TextBlob(line).sentiment.polarity) ) # TextBlob
# flairPair = lines.map(lambda line: (line, apply_fair(line))) # Flair  Excepcion en .awaitTermination() y no hay forma de solucionarlo
finBertPair = lines.map(lambda line: (line, apply_finbert(line)) ) # FinBert  Excepcion en .awaitTermination() y no hay forma de solucionarlo

result = vaderPair.join(textBlobPair)#.join(flairPair).join(finBertPair)

result.pprint()

```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```

In [72]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o4365.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\Users\gonef\anaconda3\lib\site-packages\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\Users\gonef\anaconda3\lib\site-packages\pyspark\streaming\dstream.py", line 170, in takeAndPrint
    taken = rdd.take(num + 1)
  File "C:\Users\gonef\anaconda3\lib\site-packages\pyspark\rdd.py", line 1568, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "C:\Users\gonef\anaconda3\lib\site-packages\pyspark\context.py", line 1227, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "C:\Users\gonef\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "C:\Users\gonef\anaconda3\lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    

In [68]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()